In [1]:
import getpass
import os

for key in ["OPENAI_API_KEY"]: #, "PX_SPACE_KEY", "PX_API_KEY"]:
    if not os.environ.get(key):
        print(f"Please enter key: '{key}'")
        os.environ[key] = getpass.getpass()



### Goals:
1. Very basic Zork clone
2. Function calling to play a video when a cutscene occurs
    - https://github.com/maxcurzi/tplay
    - https://github.com/joelibaceta/video-to-ascii

#### Evaluation
1. Is the right room document called up as context for each scene?
2. Does the cutscene play?

In [2]:
import nest_asyncio
import phoenix as px

nest_asyncio.apply()
px.close_app()
px.launch_app()

from phoenix.trace.langchain import LangChainInstrumentor

LangChainInstrumentor().instrument()

No active session to close
🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [3]:
import bs4
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser

from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate

from IPython.display import Video



In [4]:
text_prompt = """
### General Instructions
1. You are acting as the game engine for a text based adventure game. You are the equivalent of a D&D dungeon master.
2. The human controlling the player character will issue instructions to move their character around. 
3. You will respond in a way consistent with their previous actions and with the game state. Never acknowledge that a game is being played.

### Specific Instructions
You must:
1. Write in the second person and use the word "you" to describe the character
2. Maintain inventory management

### Inventory Rules
1. When a player picks up an object, it is added to their inventory. It remains in their inventory until it is used. 
2. Inventory is empty to start 
3. Inventory has a capacity of 10 items

### Cutscene Interactions
1. When the player touches the glowing orb in the north room a cutscene should be triggered. Very specifically, if the player says "I touch the orb" then you should
set trigger_orb_cutscene to "true"

### Description Rules
All descriptions come from room documents. You cannot make up any additional details.

{context}

User Interaction: {interaction}

### Response:

One to three sentence response to the user interaction.
""".strip()

In [5]:
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List


class Inventory(BaseModel):
    items: List[str] = Field(..., description="items in the player's inventory")
    max_capacity: int = Field(10, description="the maximum number of items the player can hold")

class GameState(BaseModel):
    inventory: Inventory = Field(..., description="the player's inventory")
    response: str = Field(..., description="your response to the player based on their latest interaction as described in the prompt")
    # room: str = Field(..., description="the room in which the character is currently in")
    trigger_orb_cutscene: bool = Field(..., description="whether an orb cutscene should occur as described in the prompt")



In [11]:
# def load_retriever():
loader = DirectoryLoader('game_data', glob="room_*", loader_cls=TextLoader)
docs = loader.load()
print(len(docs))

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})  # we only want one room to be active at a time

prompt = ChatPromptTemplate.from_template(text_prompt)
retriever = load_retriever()

llm = ChatOpenAI(model_name="gpt-4-turbo-preview", temperature=0).bind_tools([GameState])
llm_chain = (
    {"context": retriever, "interaction": RunnablePassthrough()}
    | prompt
    | llm
    | JsonOutputToolsParser()
)

llm_chain.invoke("I look around me")

2


[]

In [ ]:
class GameEngine:
    def __init__(self, llm_chain):
        self.llm_chain = llm_chain
        self.state = {}

    def input(self, text):
        state = self.llm_chain.invoke(text)
        print(state)

    def react(self):
        if self.state.get('response'):
            print(self.state['response'])

        if self.state.get('trigger_orb_cutscene'):
            Video("game_data/orb.mp4")

llm_chain = get_llm_chain()
game = GameEngine(llm_chain)

while True:
    text = input(">>> ")
    if text == "quit":
        break
    game.input(text)
    game.react()

# for text in [
#     "I look around me.",
#     "What is in my inventory?",
#     "I open the north door",
#     "I touch the orb."
# ]:
#     print(f"PROMPT: {text}")
#     print(rag_chain.invoke(text))
#     print('- - - - - - - - - - ')

In [ ]:
rag_chain.invoke("I open the door to the north")


In [ ]:
rag_chain.invoke("What is in my inventory?")